# Thicket Nsight Compute Reader: Thicket Tutorial

Nsight Compute (NCU) is a performance profiler for NVIDIA GPUs. NCU report files do not have a calltree, but with the NVTX Caliper service we can forward Caliper annotations to NCU. By profiling the same executable with a calltree profiler like Caliper, we can map the NCU data to the calltree profile and create a Thicket object. 

**NOTE: An interactive version of this notebook is available in the Binder environment.**

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/llnl/thicket-tutorial/develop)

***

## 1. Import Necessary Packages

The Thicket NCU reader requires an existing install of Nsight Compute, and the `extras/python` directory in the Nsight Compute installation directory must be added to the `PYTHONPATH`. We use `sys.path.append` to add the path to the `PYTHONPATH` in this notebook. If you are not on a Livermore Computing system, you must change this path to match your install of Nsight Compute.

In [ ]:
import sys

sys.path.append("/usr/tce/packages/nsight-compute/nsight-compute-2023.2.2/extras/python")

from IPython.display import display
from IPython.display import HTML

import thicket as tt

display(HTML("<style>.container { width:80% !important; }</style>"))

# 2. The Dataset

The dataset we are using comes from a profile of the RAJA Performance Suite on Lassen. We profile the `block_128` tuning of the `Base_CUDA` variant, while varying the problem size for 1 million and 2 million. The calltree profiles come from the CUDA Activity Profile Caliper configuration.

Below are examples of how this dataset was generated:
```
# Example run to Generate a CUDA Activity Profile
CALI_CONFIG=cuda-activity-profile,output.format=cali lrun -n 1 --smpiargs="-disable_gpu_hooks" raja-perf.exe --variants Base_CUDA --tunings block_128 --size [1048576 OR 2097152] --repfact 0.01
```

```
# Example run to Generate a NCU Report
CALI_SERVICES_ENABLE=nvtx lrun -n 1 --smpiargs="-disable_gpu_hooks" ncu --nvtx --set default --export report --metrics sm__throughput.avg.pct_of_peak_sustained_elapsed raja-perf.exe --variants Base_CUDA --tunings block_128 --size [1048576 OR 2097152] --repfact 0.01
```

In [ ]:
calltree_profile_data = [
    "../data/ncu/1M/cuda_profile.cali",
    "../data/ncu/2M/cuda_profile.cali",
]
ncu_report_data = [
    "../data/ncu/1M/report.ncu-rep",
    "../data/ncu/2M/report.ncu-rep",
]

# 3. Read Calltree Profiles into Thicket

The only performance metrics contained in the CUDA Activity Profile will be the CPU time `time` and the GPU time `time (gpu)`.

In [ ]:
tk_cap = tt.Thicket.from_caliperreader(calltree_profile_data)
tk_cap.dataframe.head(15)

# 4. Add NCU Data

The Thicket `add_ncu` function takes one required argument and one optional arguement. The required argument `ncu_report_mapping` is the mapping from the NCU report file to the corresponding calltree profile run. The optional argument `chosen_metrics` allows for a subselection of the NCU performance metrics to add, since there can be hundreds of NCU performance metrics. By default we add all metrics.

In [ ]:
# Add NCU to thicket
ncu_metrics = [
    "sm__throughput.avg.pct_of_peak_sustained_elapsed",
    "profiler__replayer_passes",
    "profiler__replayer_bytes_mem_backed_up.avg",
    "profiler__replayer_bytes_mem_backed_up.max",
    "profiler__replayer_bytes_mem_backed_up.min",
    "profiler__replayer_bytes_mem_backed_up.sum",
]
ncu_report_mapping = {
    ncu_report_data[0]: calltree_profile_data[0],
     ncu_report_data[1]: calltree_profile_data[1],
}
# Add in metrics
tk_cap.add_ncu(
    ncu_report_mapping=ncu_report_mapping, 
    chosen_metrics=ncu_metrics,
)
tk_cap.dataframe.head(15)

# 5. Add Problem Size to the Index

We can add the problem size to the performance data index for clarity about which profile we are looking at.

In [ ]:
tk_cap.metadata_column_to_perfdata("ProblemSizeRunParam")
tk_cap.dataframe = tk_cap.dataframe.reset_index().set_index(["node", "ProblemSizeRunParam"])
tk_cap.dataframe.head(15)

# 6. Visualize the NCU Performance Data on the Calltree

In [ ]:
print(tk_cap.tree(
    metric_column="sm__throughput.avg.pct_of_peak_sustained_elapsed",
    expand_name=True,
    ))